# Electric Vehicle Data Analysis Assignment

**Name:** Lekhana Gorla  
**Date:** December 22, 2025  
**Course:** MASTER DATA ANALYST - B20
---

## Introduction

The dataset contains information about **Battery Electric Vehicles (BEVs)** and **Plug-in Hybrid Electric Vehicles (PHEVs)** that are registered in **Washington State** through the Department of Licensing. It includes details such as the **vehicle make and model, model year, electric range, registration location, and eligibility for Clean Alternative Fuel Vehicle (CAFV) incentives**.  

The objective of this analysis is to:

1. **Clean and preprocess** the dataset by handling missing values, duplicates, and inconsistencies.  
2. Perform **exploratory data analysis** to identify trends, distributions, and patterns in EV adoption across Washington State.  
3. Create **visualizations** to better understand the most popular EV makes and models, regional adoption, and incentives eligibility.  
4. Build a **Linear Regression model** to predict the electric range of a vehicle based on its features.  
5. Summarize key insights and findings that could inform stakeholders about EV trends and characteristics.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display settings
pd.set_option('display.max_columns', None)


In [2]:
df = pd.read_csv("Electric_Vehicle_Population_Data.csv")

In [3]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJYGDEE8L,Thurston,Tumwater,WA,98501.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291.0,35.0,124633715,POINT (-122.89165 47.03954),PUGET SOUND ENERGY INC,5.306701e+10
1,5YJXCAE2XJ,Snohomish,Bothell,WA,98021.0,2018,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238.0,1.0,474826075,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJ3E1EBXK,King,Kent,WA,98031.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,47.0,280307233,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
3,7SAYGDEE4T,King,Issaquah,WA,98027.0,2026,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,41.0,280786565,POINT (-122.03439 47.5301),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
4,WAUUPBFF9G,King,Seattle,WA,98103.0,2016,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,43.0,198988891,POINT (-122.35436 47.67596),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10


In [4]:
df.shape

(270262, 16)

In [5]:
df.isnull().sum()

VIN (1-10)                                             0
County                                                10
City                                                  10
State                                                  0
Postal Code                                           10
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         5
Legislative District                                 649
DOL Vehicle ID                                         0
Vehicle Location                                      88
Electric Utility                                      10
2020 Census Tract                                     10
dtype: int64

In [6]:
(df['Electric Range'] == 0).sum()

np.int64(169872)

In [7]:
#Replace zeros in Electric Range
(df['Electric Range'] == df['Electric Range']).replace(0,np.nan)

0         True
1         True
2         True
3         True
4         True
          ... 
270257    True
270258    True
270259    True
270260    True
270261    True
Name: Electric Range, Length: 270262, dtype: bool

In [8]:
df.dtypes
#THERE IS NO BASE MSRP DATATYPE

VIN (1-10)                                            object
County                                                object
City                                                  object
State                                                 object
Postal Code                                          float64
Model Year                                             int64
Make                                                  object
Model                                                 object
Electric Vehicle Type                                 object
Clean Alternative Fuel Vehicle (CAFV) Eligibility     object
Electric Range                                       float64
Legislative District                                 float64
DOL Vehicle ID                                         int64
Vehicle Location                                      object
Electric Utility                                      object
2020 Census Tract                                    float64
dtype: object

In [9]:
#Fill missing Electric Range with median
median_range = df['Electric Range'].median()
df['Electric Range'].fillna(median_range)

0         291.0
1         238.0
2         220.0
3           0.0
4          16.0
          ...  
270257     21.0
270258     22.0
270259      0.0
270260      0.0
270261      0.0
Name: Electric Range, Length: 270262, dtype: float64

In [10]:
#Handle missing geographic columns
df['County'].fillna('Unknown')
df['City'].fillna('Unknown')
df['Postal Code'].fillna(0)
df['Electric Utility'].fillna('Unknown')

0                                    PUGET SOUND ENERGY INC
1                                    PUGET SOUND ENERGY INC
2             PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
3             PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
4              CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA)
                                ...                        
270257        PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
270258    BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...
270259    BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...
270260                               PUGET SOUND ENERGY INC
270261                            PUD NO 1 OF CHELAN COUNTY
Name: Electric Utility, Length: 270262, dtype: object

In [11]:
df['Vehicle Location'].fillna('0,0')

0         POINT (-122.89165 47.03954)
1          POINT (-122.18384 47.8031)
2         POINT (-122.17743 47.41185)
3          POINT (-122.03439 47.5301)
4         POINT (-122.35436 47.67596)
                     ...             
270257    POINT (-122.60761 47.08798)
270258    POINT (-123.14135 47.40639)
270259    POINT (-122.52082 47.26887)
270260     POINT (-122.18384 47.8031)
270261    POINT (-120.30521 47.41493)
Name: Vehicle Location, Length: 270262, dtype: object

In [12]:
df['Legislative District'].fillna(df['Legislative District'].median())
df['2020 Census Tract'].fillna(df['2020 Census Tract'].median())

0         5.306701e+10
1         5.306105e+10
2         5.303303e+10
3         5.303302e+10
4         5.303300e+10
              ...     
270257    5.305307e+10
270258    5.304596e+10
270259    5.305306e+10
270260    5.306105e+10
270261    5.300796e+10
Name: 2020 Census Tract, Length: 270262, dtype: float64

In [13]:
#Remove Duplicate Rows
df.drop_duplicates(subset=['VIN (1-10)', 'DOL Vehicle ID'])

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJYGDEE8L,Thurston,Tumwater,WA,98501.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291.0,35.0,124633715,POINT (-122.89165 47.03954),PUGET SOUND ENERGY INC,5.306701e+10
1,5YJXCAE2XJ,Snohomish,Bothell,WA,98021.0,2018,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,238.0,1.0,474826075,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJ3E1EBXK,King,Kent,WA,98031.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,47.0,280307233,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
3,7SAYGDEE4T,King,Issaquah,WA,98027.0,2026,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,41.0,280786565,POINT (-122.03439 47.5301),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
4,WAUUPBFF9G,King,Seattle,WA,98103.0,2016,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,43.0,198988891,POINT (-122.35436 47.67596),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270257,1C4RJXN60R,Pierce,Joint Base Lewis Mcchord,WA,98433.0,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,28.0,266021122,POINT (-122.60761 47.08798),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.305307e+10
270258,1C4JJXR66N,Mason,Hoodsport,WA,98548.0,2022,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,22.0,35.0,282482938,POINT (-123.14135 47.40639),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,5.304596e+10
270259,7SAYGDEEXP,Pierce,Tacoma,WA,98406.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,27.0,228485085,POINT (-122.52082 47.26887),BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,5.305306e+10
270260,5YJYGDEE2M,Snohomish,Bothell,WA,98021.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,1.0,282699217,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10


In [14]:
import hashlib
df['VIN (1-10)'].apply(
    lambda x: hashlib.sha256(x.encode()).hexdigest()
)

0         f6c53bc06f819ee23fea90e181292f8a1622fda6b6ab7b...
1         6222905c9f774b0ae8a03d6c26eb848ac28255d9f618f2...
2         78953a9f9d62e8cc12a944c5a3c1e08a4d3e1b55a9759e...
3         9a118e60068455f53c5a714941d31f1d6728d2b8ce8d10...
4         dbd150d32969e963691fa8b3eb06fcabd516f97fcb8078...
                                ...                        
270257    bb81ca3ffcef417e7d219e553ca374c05d8d14b8a9ddf4...
270258    479588897e46b699e9765483702f414961c674262b8e9b...
270259    c3da8f30157c9a0b41ea5cd43dd0de1ca0c77ed25c2f37...
270260    b0f842a0c3d5e36db2b7c4f7a78167b317aaa113945171...
270261    26936defe33efb04f9bcf4fc4a6587b19121308eee14d7...
Name: VIN (1-10), Length: 270262, dtype: object

In [45]:
# Extract coordinates and assign to new columns
df[['Longitude', 'Latitude']] = df['Vehicle Location'].str.extract(
    r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)'
)

# Convert them to float
df['Longitude'] = df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)

In [ ]:
df[['Vehicle Location', 'Longitude', 'Latitude']].head()


In [ ]:
#Top 5 EV Makes
Top_makes = df['Make'].value_counts().head()
print(Top_makes)
#Top 5 EV Models
Top_models = df['Model'].value_counts().head()
print(Top_models)

In [ ]:
#EV Distribution by County
county_counts = df['County'].value_counts().head()
print(county_counts)

In [ ]:
#EV Adoption Over Model Years
year_counts = df.groupby('Model Year').size()
print(year_counts)

In [43]:
#Average Electric Range
avg_range= df['Electric Range'].mean()
print(f"Average Electric Range:{avg_range:.2f} miles")

Average Electric Range:40.39 miles


In [ ]:
cafv_counts = df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].value_counts(normalize=True) * 100
print(cafv_counts)

In [ ]:
#How does the electric range vary across different makes and models
range_by_make = df.groupby('Make')['Electric Range'].mean().sort_values(ascending=False)
print(range_by_make)
range_by_model = df.groupby('Model')['Electric Range'].mean().sort_values(ascending=False)
print(range_by_model.head(10))  # Top 10 models by range

In [ ]:
#Regional Trends (Urban vs Rural Areas)
urban_counties = ['King', 'Snohomish', 'Pierce', 'Clark', 'Spokane']
df['Region'] = df['County'].apply(lambda x: 'Urban' if x in urban_counties else 'Rural')
df['Region'].value_counts()

In [ ]:
#DATA VISUALIZATION
#Bar Chart – Top 5 EV Makes
Top_makes.plot(kind = 'bar', color = 'skyblue', title = 'Top 5 EV Makes')
plt.xlabel('Make')
plt.ylabel('Count')
plt.show()
#Bar Chart – Top 5 EV Models
Top_models.plot(kind='bar', color='lightgreen', title='Top 5 EV Models')
plt.xlabel('Model')
plt.ylabel('Count')
plt.show()

In [ ]:
!pip install geopandas matplotlib
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
county_counts = df['County'].value_counts().reset_index()
county_counts.columns = ['County', 'EV_Count']
wa_counties = gpd.read_file(
    "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
)
wa_counties = wa_counties[wa_counties['STATE'] == '53']
wa_counties['County'] = wa_counties['NAME']
wa_map = wa_counties.merge(
    county_counts,
    on='County',
    how='left'
)

wa_map['EV_Count'] = wa_map['EV_Count'].fillna(0)
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

wa_map.plot(
    column='EV_Count',
    cmap='OrRd',
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    legend=True
)

ax.set_title("Electric Vehicle Distribution by County in Washington State")
ax.axis('off')

plt.show()

In [ ]:
year_counts.plot(kind='line', color='purple', marker='o', title='EV Adoption Trend by Model Year')
plt.xlabel('Model Year')
plt.ylabel('Number of EVs')
plt.show()

In [ ]:
plt.scatter(df['Model Year'], df['Electric Range'],alpha=0.5)
plt.title('Electric Range vs Model Year')
plt.xlabel('Model Year')
plt.ylabel('Electric Range')
plt.show()

In [ ]:
#Pie Chart – CAFV Eligibility
cafv_counts.plot(kind='pie', autopct='%1.1f%%', title='CAFV Eligibility')
plt.ylabel('')
plt.show()

In [ ]:
!pip install folium
import folium

In [ ]:
df[['Latitude', 'Longitude']].isnull().sum()

In [ ]:
df_map = df.dropna(subset=['Latitude', 'Longitude'])

In [ ]:
# Initialize map
ev_map = folium.Map(location=[47.6062, -122.3321], zoom_start=7)

# Plot first 1000 points with valid coordinates
for idx, row in df_map.head(1000).iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(ev_map)

# Save map
ev_map.save("EV_Map.html")

In [ ]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [35]:
# Fill missing Electric Range with median
median_range = df['Electric Range'].median()
df['Electric Range'].fillna(median_range, inplace=True)

# Verify
df['Electric Range'].isnull().sum()  # Should now be 0


np.int64(0)

In [36]:
#Handle Categorical Variables
# Select features
features = ['Model Year', 'Make', 'Model', 'Electric Vehicle Type']

# One-hot encode categorical features
X = pd.get_dummies(df[features], drop_first=True)

# Target variable
y = df['Electric Range']

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [39]:
#Fit Linear Regression Model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(f"R² score: {r2:.3f}")

R² score: 0.518


In [40]:
#Interpret Coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

coefficients.head(10)

,Feature,Coefficient
102,Model_ESCAPE,130.794162
148,Model_MACAN,102.774165
50,Model_550E,100.283801
182,Model_RAV4 PRIME (PHEV),95.319804
70,Model_BOLT EV,94.750508
88,Model_E-CLASS,85.036240
147,Model_M5,84.870569
54,Model_750E,81.290116
118,Model_GLE-CLASS,71.181813
226,Model_XM,71.050976


#Improving Model Accuracy

Include more features (battery size, weight, kWh, MSRP if available)

Remove outliers

Try Polynomial Regression, Random Forest, or Gradient Boosting

Feature scaling (optional for regularization models)

In [41]:
#Predicting New EV Models
new_ev = pd.DataFrame({
    'Model Year': [2025],
    'Make': ['Tesla'],
    'Model': ['Model Z'],
    'Electric Vehicle Type': ['BEV']
})

# One-hot encode to match training features
new_ev_encoded = pd.get_dummies(new_ev)
new_ev_encoded = new_ev_encoded.reindex(columns=X.columns, fill_value=0)

predicted_range = model.predict(new_ev_encoded)
print(f"Predicted Electric Range: {predicted_range[0]:.2f} miles")

Predicted Electric Range: -13.74 miles


## Conclusion

This analysis of Washington State electric vehicle registrations provided several key insights:

1. **Popular EV Makes and Models:** Tesla, Hyundai, and Kia are the most commonly registered brands, with models like Model 3 and Ioniq 5 leading the registrations.  
2. **Regional Trends:** King County has the highest number of EV registrations, indicating higher adoption in urban areas.  
3. **EV Adoption Over Time:** The number of EVs has steadily increased over recent model years, reflecting growing awareness and incentive programs.  
4. **Electric Range:** The average electric range of EVs is approximately 40.39 miles. Battery electric vehicles (BEVs) generally have higher ranges compared to plug-in hybrids (PHEVs).  
5. **CAFV Eligibility:** A significant percentage of vehicles are eligible for Clean Alternative Fuel Vehicle incentives, supporting clean energy adoption.  
6. **Regression Model Insights:** The Linear Regression model demonstrated that newer model years and specific makes/models influence electric range. While the model explains part of the variance (R² = 0.518), further features such as battery capacity or MSRP could improve prediction accuracy.  

Overall, this analysis highlights **trends in EV adoption, regional differences**
